Intiate haystack database

In [1]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

In [2]:
document_store = ElasticsearchDocumentStore(host='localhost', username='', password='', index='squad_docs')

In [3]:
import requests
res = requests.get('http://localhost:9200/_cluster/health')
res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 4,
 'active_shards': 4,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 66.66666666666666}

In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)



In [5]:
import pickle
  
with open('nlp_transformer', 'wb') as f:
 
  # Step 3
  pickle.dump(nlp, f)

In [6]:
from haystack.retriever.sparse import BM25Retriever

In [7]:
retriever = BM25Retriever(document_store)

In [9]:
query = "What is hatsune miku?"

In [10]:
responses = retriever.retrieve(query)

In [11]:
responses

[<Document: {'content': 'Miku\'s popularity has resulted in various references to her in anime. Miku is the protagonist of a manga series named Maker Hikōshiki Hatsune Mix, written by Kei Garō. The manga explores the many possibilities of story-telling and has featured numerous adventures, ranging from giant-sized battles with Hatsune Miku to home exploits. There is therefore no single storyline, and the entire setting within the manga is unofficial. During an episode of Zoku Sayonara, Zetsubou Sensei, Miku is seen auditioning for the voice of Meru Otonashi (Kagamine Rin and Len are referenced in the same episode). Miku\'s voice is used in one of the ending themes for the anime series Akikan! (episode 12). Moreover, she also sings the ending theme for the anime Yamishibai: Japanese Ghost Stories, called "Kaikai Emaki" (怪々絵巻). During an episode in the Lucky Star OVA, Kagami Hiiragi gets magically transformed into Miku cosplay. A character in the anime Kämpfer appears dressed as Miku in 

In [12]:
answers = []

for response in responses:
    QA_input = {
        'question': query,
        'context': response.to_dict()['content']
    }
    
    answers.append(nlp(QA_input))
    

In [13]:
answers

[{'score': 0.21621721982955933,
  'start': 118,
  'end': 145,
  'answer': 'Maker Hikōshiki Hatsune Mix'},
 {'score': 0.34318476915359497,
  'start': 956,
  'end': 979,
  'answer': 'original concept design'},
 {'score': 0.2849087715148926,
  'start': 162,
  'end': 172,
  'answer': 'an android'},
 {'score': 0.05226878076791763,
  'start': 276,
  'end': 285,
  'answer': 'Snow Miku'},
 {'score': 0.1977575570344925,
  'start': 131,
  'end': 148,
  'answer': 'Crypton Vocaloids'},
 {'score': 0.009457817301154137,
  'start': 288,
  'end': 305,
  'answer': 'B Who I Want 2 B"'},
 {'score': 0.0013418395537883043,
  'start': 204,
  'end': 211,
  'answer': 'angle θ'},
 {'score': 0.07147035002708435,
  'start': 578,
  'end': 609,
  'answer': 'giving inanimate objects a soul'},
 {'score': 0.505394458770752,
  'start': 92,
  'end': 121,
  'answer': 'a Vocaloid software voicebank'},
 {'score': 0.06241319328546524, 'start': 97, 'end': 105, 'answer': 'due east'}]

In [104]:
finalAnswer = max(answers, key=lambda x:x['score'])

In [105]:
finalAnswer

{'score': 0.525010883808136,
 'start': 4,
 'end': 41,
 'answer': 'six important trigonometric functions'}

In [117]:
while True:
    query = input()
    responses = retriever.retrieve(query)
    
    answers = []

    for response in responses:
        QA_input = {
            'question': query,
            'context': response.to_dict()['content']
        }

        answers.append(nlp(QA_input))
    
    finalAnswer = max(answers, key=lambda x:x['score'])['answer']
    
    print(finalAnswer)
    

KeyboardInterrupt: Interrupted by user